# Introduction

In this quickstart, we will get Data Detective running on your dataset as quickly as possible.

To get up and running on your own dataset as quickly as possible, we have formatted the tutorial so that sections requiring your own code / inputs are **bolded**; all other sections can be customized as needed, but can be skimmed / run as-is. 
 


In [1]:
import numpy as np
import pandas as pd
import time
import torch

from typing import Dict, Union

from constants import FloatTensor
from src.data_detective_engine import DataDetectiveEngine
from src.enums.enums import DataType

/Users/mcconnl3/Code/data-detective-load-test/dd_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mcconnl3/Code/data-detective-load-test/dd_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## **Step 1: Dataset Implementation**

### **Option 1: CSV Dataset Example**

The easiest way to get started with Data Detective for CSV data is to use the `CSVDataset` class. This class accepts the path for a CSV file as well as a dictionary containing the datatypes for each column in the CSV file. 

The CSV file can contain numbers, text, or images represented in the CSV file as absolute paths. The datatype options available in the CSV Dict include: 
- `DataType.CONTINUOUS`
- `DataType.MULTIDIMENSIONAL` 
- `DataType.CATEGORICAL` 
- `DataType.TEXT`
- `DataType.IMAGE`
- `DataType.SEQUENTIAL`

If it suits your use case, fill in the blank code is available below to create the CSVDataset below. Otherwise, skip to `Dataset Construction` to find out how to build your own dataset.




In [2]:
from src.datasets.csv_dataset import CSVDataset

dataset = CSVDataset(
    # change filepath to your csv filepath
    filepath="your_csv_filepath.csv",
    # change dictionary to map from csv filenames to data types
    datatypes={
        "column1": DataType.CONTINUOUS,
        "column2": DataType.MULTIDIMENSIONAL,
        # ...
        "column_k": DataType.IMAGE,
    }
)

TypeError: __init__() got an unexpected keyword argument 'datatypes'

Note: if there is an `IMAGE` column in the CSV dataset that contains image paths, they will automatically be loaded into the dataset via `np.load`. 

### **Option 2: Dataset Construction**

If dealing with data that does not easily serialize in CSV format, it is easier to create your own dataset to work within the Data Detective framework. Your dataset needs to satisfy the following requirements: 

1. It must override the `__getitem__` method that returns a dictionary mapping from each data column key to the data value. 
2. It must contain a `datatypes` method that returns a dictionary mapping from each data column key to the column's datatype. 
3. It must inherit from `torch.utils.data.DataType`.
4. \[optional\] It is convenient, but not necessary, to define a `__len__` method. `


Before diving in, let's look at a very simple dataset that consists of 10 columns of normal random variables. 

In [2]:
from pandas import DataFrame
from torch.utils.data import Dataset

from src.datasets.data_detective_dataset import DataDetectiveDataset

class NormalDataset(DataDetectiveDataset):
    def __init__(self, num_cols: int = 10, dataset_size: int = 1000, loc: float = 0.):
        """
        Creates a normal dataset with column `feature_k` for k in [0, num_cols) 
        @param num_cols: number of columns to have
        @param dataset_size: number of datapoints to have
        @param loc: the mean of the data. 
        """
        self.dataset_size = dataset_size
        self.columns = [f"feature_{j}" for j in range(num_cols)]

        dataframe: DataFrame = pd.DataFrame({
            f"feature_{i}": np.random.normal(loc, 100, size=dataset_size)
            for i in range(num_cols)
        }, columns=self.columns)

        self.dataframe = dataframe
        
        super().__init__(
            show_id=False, 
            include_subject_id_in_data=False,
            sample_ids = [str(s) for s in list(range(dataset_size))],
            subject_ids = [str(s) for s in list(range(dataset_size))]
        )

    def __getitem__(self, index: int) -> Dict[str, float]:
        """
        Returns a dict containing each column mapped to its value. 
        """
        return self.dataframe.iloc[index].to_dict()

    def __len__(self):
        return self.dataset_size

    def datatypes(self) -> Dict[str, DataType]:
        """
        Returns a dictionary mapping each column to its datatype.
        """
        return {
            column_name: DataType.CONTINUOUS
            for column_name in self.columns
        }


class BrainStudy(torch.utils.data.Dataset):
    def __init__(self, pathlist=None, **kwargs):
        super().__init__(**kwargs)
        self.df = pd.read_csv(pathlist) 

    def datatypes(self) -> Dict[str, DataType]:
        """
        Specify datatype for each column name. 
            # SEQUENCE
            # TEXT
            # IMAGE
            # CONTINUOUS
            # CATEGORICAL
            # MULTIDIMENSIONAL  vs  MULTIVARIATE vs VECTOR
        
            DataType.CONTINUOUS
            DataType.CATEGORICAL
            DataType.MULTIDIMENSIONAL  # maybe replace with MULTIVARIATE
            DataType.IMAGE  # maybe replace with GRID
            DataType.TIME_SERIES  # maybe replace with SEQUENCE

            okay what needs to happen to make this semi-working?
                - age: depictable by normal distribution or something.
                - sex: 50/50
                - cognitive score: uniform random normal
                - clin history?
        """
        self.datatype_dict =  {
            "age": DataType.CONTINUOUS,
            "sex": DataType.CATEGORICAL,
            "cognitive_score": DataType.CONTINUOUS,
            "clinical_history": DataType.TEXT,#path to .txt?
            "brain_MRI": DataType.IMAGE, #path
            "brain_PET": DataType.IMAGE,#path
            "activity_monitor": DataType.SEQUENCE,#path
            "speech_derived_features": DataType.MULTIVARIATE,#path to .npy ?
        }
        return self.datatype_dict    
        
    def __getitem__(self, idx: Union[int, slice, list]) -> Dict[str, Union[FloatTensor, int]]:
        """
        Returns a dictionary with column names and values for a specific idx. 
        """
        # for data inputs that are paths, code to load the file from the path needs to be provided in getitem
        # code to load in sequence, image, or multivariate data
        return {k:self.df[k][idx] for k in self.datatype_dict.keys()}


# data_object = BrainStudy(pathlist='/path/to/my/pathlist.csv')

# what it would look like for several splits
# data_object = {'train': BrainStudy(pathlist='/path/to/my/train.csv'),
#                'val':BrainStudy(pathlist='/path/to/my/val.csv'),
#                'internal_test':BrainStudy(pathlist='/path/to/my/internal_test.csv'),}


data_detective_engine = DataDetectiveEngine()

dataset = NormalDataset() 

Above, you can see that the dataset has both of the requirements above:

1. It overrides `__getitem__` to provide a dict mapping from each column to a single value. 
2. It overrides `datatypes` to map the same keys in `__getitem__` to their datatypes. 
3. It inherits from `torch.utils.data.Dataset`.

For complete clarity, let's take a look at the outputs of (1) and (2) below: 


In [ ]:
dataset.__getitem__(0)

{'feature_0': -0.7755474851575616,
 'feature_1': -0.01171192123816041,
 'feature_2': 0.15464464394518335,
 'feature_3': -2.4529767374596907,
 'feature_4': -1.564522897681607,
 'feature_5': -0.45129237860828064,
 'feature_6': -0.7442126446179803,
 'feature_7': 0.3120126662207676,
 'feature_8': -1.4746812039195702,
 'feature_9': -0.9357401007015353}

In [ ]:
dataset.datatypes()

{'feature_0': <DataType.CONTINUOUS: 'continuous'>,
 'feature_1': <DataType.CONTINUOUS: 'continuous'>,
 'feature_2': <DataType.CONTINUOUS: 'continuous'>,
 'feature_3': <DataType.CONTINUOUS: 'continuous'>,
 'feature_4': <DataType.CONTINUOUS: 'continuous'>,
 'feature_5': <DataType.CONTINUOUS: 'continuous'>,
 'feature_6': <DataType.CONTINUOUS: 'continuous'>,
 'feature_7': <DataType.CONTINUOUS: 'continuous'>,
 'feature_8': <DataType.CONTINUOUS: 'continuous'>,
 'feature_9': <DataType.CONTINUOUS: 'continuous'>}

Note that both dictionaries contain identical keys, indicating that no datatypes are missed in the definition of the `datatypes` function. 

Below is the skeleton code for a dataset construction. Fill it in with your desired implemenetation of `__getitem__` and `datatypes`, and any initialization you may need to do.  

In [ ]:
class YourDataset(Dataset):
    def __init__(self):
        """
        Sets up the dataset. This can include steps like:
            - loading csv paths
            - reading in text data
            - cleaning and preprocessing
        """
    
        """
        YOUR CODE HERE
        PUT YE CODE HERE, MATEY
        ARR
        """

    def __getitem__(self, index: int) -> Dict[str, float]:
        """
        Returns a dict containing each column mapped to its value. 
        """
    
        """
        YOUR CODE HERE
        AHOY, YE SCURVY CODER! WRITE YER MAGIC HERE!
        """


        return self.dataframe.iloc[index].to_dict()

    def datatypes(self) -> Dict[str, DataType]:
        """
        Returns a dictionary mapping each column to its datatype.
        """

        """
        YOUR CODE HERE
        AHOY, YE SCURVY CODER! WRITE YER MAGIC HERE!
        """

    # NOTE: convenient, but not optional, to add __len__ method
    # def __len__(self) -> int: 
    #     pass

# put initialization code here or fix if needed
dataset = YourDataset()

Now that you've written your dataset, lets make sure everything is in ship shape!

In [ ]:
dataset[0]

AttributeError: 'YourDataset' object has no attribute 'dataframe'

In [ ]:
dataset.datatypes()

{'feature_0': <DataType.CONTINUOUS: 'continuous'>,
 'feature_1': <DataType.CONTINUOUS: 'continuous'>,
 'feature_2': <DataType.CONTINUOUS: 'continuous'>,
 'feature_3': <DataType.CONTINUOUS: 'continuous'>,
 'feature_4': <DataType.CONTINUOUS: 'continuous'>,
 'feature_5': <DataType.CONTINUOUS: 'continuous'>,
 'feature_6': <DataType.CONTINUOUS: 'continuous'>,
 'feature_7': <DataType.CONTINUOUS: 'continuous'>,
 'feature_8': <DataType.CONTINUOUS: 'continuous'>,
 'feature_9': <DataType.CONTINUOUS: 'continuous'>}

In [ ]:
assert(isinstance(dataset[0], dict))
assert(isinstance(dataset.datatypes(), dict))
assert(dataset[0].keys() == dataset.datatypes().keys())

# Step 2: Data Object Creation

The *data object* is a dictionary that consists of the preprocessed dataset and (optionally) its splits. More information about setting up the data object is available in the [main tutorial](Tutorial.ipynb) and the [ExtendingDD tutorial](ExtendingDD.ipynb).; for the purpose of the quickstart, splitting and organization is done for you. 

In [3]:
from src.datasets.data_detective_dataset import dd_random_split


inference_size: int = 20
everything_but_inference_size: int = dataset.__len__() - inference_size
inference_dataset, everything_but_inference_dataset = dd_random_split(dataset, [inference_size, dataset.__len__() - inference_size])
    
train_size: int = int(0.6 * len(everything_but_inference_dataset))
val_size: int = int(0.2 * len(everything_but_inference_dataset))
test_size: int = len(everything_but_inference_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = dd_random_split(everything_but_inference_dataset, [train_size, val_size, test_size])

data_object = {
    "entire_set": dataset,
    "everything_but_inference_set": everything_but_inference_dataset,
    "inference_set": inference_dataset,
    # unordered splits belong here
    # in this example, train/val/test are included, but this section can be as long
    # as desired and can contain an arbitrary number of named splits 
    "train/val/test": {
        "training_set": train_dataset,
        "validation_set": val_dataset,
        "test_set": test_dataset,
    },
    # Example of k-fold split:
    # "fold_0": {
    #      "training_set": train_datasets[0],
    #      "test_set": test_datasets[0],
    # },
    # "fold_1": {
    #      "training_set": train_datasets[1],
    #      "test_set": test_datasets[1],
    # },
    # ...
    # "fold_k": {
    #      "training_set": train_datasets[j],
    #      "test_set": test_datasets[j],
    # }
}

print(f"size of inference_dataset: {inference_dataset.__len__()}")
print(f"size of everything_but_inference_dataset: {everything_but_inference_dataset.__len__()}")
print(f"size of train_dataset: {train_dataset.__len__()}")
print(f"size of entire dataset: {dataset.__len__()}")
print(f"size of val_dataset: {val_dataset.__len__()}")
print(f"size of test_dataset: {test_dataset.__len__()}")

size of inference_dataset: 20
size of everything_but_inference_dataset: 980
size of train_dataset: 588
size of entire dataset: 1000
size of val_dataset: 196
size of test_dataset: 196


# Step 3: Setting up a Validation Schema

## Step 3.1: Specifying Validators and Options

The validation schema contains information about the types of checks that will be executed by the Data Detective Engine and the transforms that Data Detective will use. More detailsd about creating your own validation schema is available in the [main tutorial](Tutorial.ipynb); below is the validation schema that we recommend to get started. 

In [4]:
validation_schema : Dict = {
    "validators": {
        # "unsupervised_anomaly_data_validator": {},
        "unsupervised_multimodal_anomaly_data_validator": {},
        # "split_covariate_data_validator": {},
        # "ood_inference_data_validator": {}
    }
}

## Step 3.2: Specifying Transforms

It may be the case that you are using a data modality that has little to no method infrastructure in Data Detective. The simplest way to make use of all of Data Detective's functionality is to use a transform that maps this data modality to a well-supported modality in Data Detective such as multidimensional data. In our example, we will be making use of a pretrained resnet50 backbone to map images to 2048 dimensional vectors. This will allow us to make use of methods built for multidimensional data on our image representations. 

More information about introducing custom transforms into Data Detective and customizing the transform schema with pre-existing transforms is available in the [main tutorial](Tutorial.ipynb) and explanations on how to create/use your own transforms are available in the [ExtendingDD tutorial](ExtendingDD.ipynb).


In [5]:
transform_schema : Dict = {
    "transforms": {
        "IMAGE": [{
            "name": "resnet50",
            "in_place": "False",
            "options": {},
        }],
    }
}
     
full_validation_schema: Dict = {
    **validation_schema, 
    **transform_schema
}

# Step 4: Running the Data Detective Engine

Now that the full validation schema and data object are prepared, we are ready to run the Data Detective Engine.

In [6]:
data_detective_engine = DataDetectiveEngine()

start_time = time.time()
results = data_detective_engine.validate_from_schema(full_validation_schema, data_object)
print("--- %s seconds ---" % (time.time() - start_time))

Cache loaded from data/tmp/cache.pkl
running validator class unsupervised_multimodal_anomaly_data_validator...
thread 11038273536 entered to handle validator method iforest_multimodal_anomaly_validator_method
thread 11038273536:    running iforest_multimodal_anomaly_validator_method...
thread 11038273536: finished
thread 11071926272 entered to handle validator method cblof_multimodal_anomaly_validator_method
thread 11071926272:    running cblof_multimodal_anomaly_validator_method...


/Users/mcconnl3/Code/data-detective-load-test/dd_env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/mcconnl3/Code/data-detective-load-test/dd_env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/mcconnl3/Code/data-detective-load-test/dd_env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/mcconnl3/Code/data-detective-load-test/dd_env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 

thread 11071926272: finished
thread 11199885312 entered to handle validator method pca_multimodal_anomaly_validator_method
thread 11199885312:    running pca_multimodal_anomaly_validator_method...
thread 11199885312: finished
Cache written to data/tmp/cache.pkl
--- 8.35102105140686 seconds ---


In [7]:
results

{'unsupervised_multimodal_anomaly_data_validator': {'iforest_multimodal_anomaly_validator_method': {'results': {'0': -0.00386062744083715,
    '1': -0.046648734344571896,
    '2': -0.0033060802957304847,
    '3': -0.05107521932633258,
    '4': -0.10052378826977659,
    '5': -0.0408631968216267,
    '6': -0.015948852771423938,
    '7': -0.062154963347270276,
    '8': -0.039812094772213535,
    '9': -0.03735364890441717,
    '10': -0.07926071047411515,
    '11': -0.022002611401122307,
    '12': -0.0166187690494376,
    '13': -0.036585685479440744,
    '14': 0.02284067291215003,
    '15': -0.08112817376614173,
    '16': -0.028304255657208344,
    '17': -0.03429735067806183,
    '18': -0.07015980332536853,
    '19': -0.07967432222165027,
    '20': -0.050896903997499854,
    '21': -0.037072826158776495,
    '22': -0.004107437632660249,
    '23': -0.08119939130037113,
    '24': -0.0880412714138804,
    '25': -0.057727002172815955,
    '26': -0.004124700469908149,
    '27': -0.097841577612339

Great! Let's start to look at and analyze the results we've collected.

# Step 5: Interpreting Results using the Built-In Rank Aggregator

To do rank aggregation, create a rankings object and either aggregate completely with the `aggregate_results_modally` or aggregate by a single modality with the `aggregate_results_multimodally`. See below 

In [8]:
from enum import Enum

import pandas as pd
import scipy
from typing import List

from src.aggregation.rankings import RankingAggregationMethod, ResultAggregator

aggregator = ResultAggregator(results_object=results)
# modal_rankings = aggregator.aggregate_results_modally("unsupervised_anomaly_data_validator", [RankingAggregationMethod.LOWEST_RANK, RankingAggregationMethod.HIGHEST_RANK, RankingAggregationMethod.ROUND_ROBIN], given_data_modality="feature_0")
total_rankings = aggregator.aggregate_results_multimodally("unsupervised_multimodal_anomaly_data_validator", [RankingAggregationMethod.LOWEST_RANK, RankingAggregationMethod.HIGHEST_RANK])
total_rankings

,results_iforest_multimodal_anomaly_validator_method_rank,results_cblof_multimodal_anomaly_validator_method_rank,results_pca_multimodal_anomaly_validator_method_rank,lowest_rank_agg_rank,highest_rank_agg_rank,results_iforest_multimodal_anomaly_validator_method_score,results_cblof_multimodal_anomaly_validator_method_score,results_pca_multimodal_anomaly_validator_method_score
0,116,125,167,108,169,-0.003861,346.030280,394.258062
1,490,631,508,542,587,-0.046649,244.184598,320.792766
10,858,856,782,794,867,-0.079261,199.961117,272.767473
100,372,174,208,285,231,-0.036181,330.098858,383.948859
101,15,63,8,33,13,0.044518,374.058180,504.554893
...,...,...,...,...,...,...,...,...
995,705,799,666,717,759,-0.065646,214.757442,293.908632
996,715,697,648,624,744,-0.066272,233.596599,297.798324
997,808,810,903,852,883,-0.073051,211.944142,241.808826
998,400,254,231,324,300,-0.038926,306.271459,376.012882


### Appendix 1A: Complete list of validator methods

| name | path | method description | data types | operable split types | 
| ---- | ---- | ------------------ | ---------- | -------------------- | 
| adbench_validator_method | src/validator_methods/validator_method_factories/adbench_validator_method_factory.py | factory generating adbench methods that perform anomaly detection | multidimensional data | entire set | 
| adbench_multimodal_validator_method | src/validator_methods/validator_method_factories/adbench_multimodal_validator_method_factory.py | factory generating adbench methods that perform anomaly detection by concatenating all multidimensional columns first to be able to draw conclusions jointly from the data | multidimensional data | entire set | 
| adbench_ood_inference_validator_method | src/validator_methods/validator_method_factories/adbench_ood_inference_validator_method_factory.py | factory generating methods that perform ood testing given a source set and a target/inference set using adbench emthods | multidimensional data | inference_set, everything_but_inference_set | 
| chi square validator method | src/validator_methods/chi_square_validator_method.py | chi square test for testing CI assumptions between two categorical variables | categorical data | entire_set |
| diffi anomaly explanation validator method | src/validator_methods/diffi_anomaly_explanation_validator_method.py | A validator method for explainable anomaly detection using the DIFFI feature importance method. | multidimensional | entire_set |
| fcit validator method | src/validator_methods/fcit_validator_method.py | A method for determining conditionanl independence of two multidimensional vectors given a third. | continuous, categorical, or multidimensional | entire_set |
| kolmogorov smirnov multidimensional split validator | src/validator_methods/kolmogorov_smirnov_multidimensional_split_validator_method.py | KS testing over multidimensional data for split covariate shift. | multidimensional | entire_set |
| kolmogoriv smirnov normality validator method | src/validator_methods/kolmogorov_smirnov_normality_validator_method.py | KS testing over continuous data for normality assumption. | continuous | entire_set | 
| kolmogorov smirnov split validator method | src/validator_methods/kolmogorov_smirnov_split_validator_method.py | KS testing over continuous data for split covariate shift. |  continuous | entire_set |  
| kruskal wallis multidimensional split validator method | src/validator_methods/kruskal_wallis_multidimensional_split_validator_method.py | kruskal wallis testing over multidimensional data for split covariate shift. | multidimensional | entire_set | 
| kruskal wallis split validator method | src/validator_methods/kruskal_wallis_split_validator_method.py | kruskal wallis testing over continuous data for split covariate shift. | continuous | entire_set |  
| mann whitney multidimensional split validator method | src/validator_methods/mann_whitney_multidimensional_split_validator_method.py | mann whitney testing over multidimensional data for split covariate shift. | multidimensional | entire_set |
| mann whitney split validator method | src/validator_methods/mann_whitney_split_validator_method.py | mann whitney testing over continuous data for split covariate shift. | continuous | entire_set |  
| shap tree validator method | src/validator_methods/shap_tree_validator_method.py |     A validator method for explainable anomaly detection using Shapley values. | multidimensional | entire_set | 

